---
title: 极客时间-现代C++30讲 02丨自己动手，实现C++的智能指针
tags: 小书匠,C++,极客时间-现代C++30讲,raii,heap,free store,new,malloc,delete,free,smart_ptr,auto_ptr,share_ptr,unique_ptr,智能指针,pointer|指针
grammar_cjkRuby: true
---

[toc]

# 02丨自己动手，实现C++的智能指针

## Demands on smart pointers

1. Can be used for arbitrary class.
2. Act like normal pointers.
3. Support copy.

## Use template

```cpp
template<class T>
class smart_pointer
{
    public:
        smart_pointer(T* ptr): ptr_(ptr) { }
        ~smart_pointer()
        {
            delete ptr_;
        }
        T* get() const { return ptr_; }
    private:
        T* ptr_;
    
};
```

## Act like normal pointers

How do normal pointers act? Basically, normal pointers support 3 opeartions on them

1. use `ptr->x` to access a member
2. use `*ptr` to get the reference of the pointed object
3. can be used as a boolean variable, such as `if(ptr) ptr->call()`
4. A pointer to derived class can be converted to a pointer to base class.

### overwrite operations

Thus, we can overwrite these operations to make `smart_pointer` more like a normal pointers.

```cpp
template<class T>
class smart_pointer
{
    public:
        smart_pointer(T* ptr): ptr_(ptr) { }
        ~smart_pointer()
        {
            delete ptr_;
        }
        T* get() const { return ptr_; }
        // Operations to immitate pointers
        T& operator*() const { return *ptr_; }
        T* operator->() const { return ptr_;  }
        operator bool() const { return ptr_; }
    private:
        T* ptr_;
    
};
```

### use template function

Suppose our class support copy constructor. What the signature of the copy constructor? It might be

```cpp
smart_pointer(const smart_point<T>&);
```

However, this signature doesn't meet our demands, because it doesn't support accepts a derived class of T.

To support convert a derived class pointer into a base class pointer, we should use template function.

```cpp
template<class U>
smart_pointer(const smart_point<U>&);
```

## Three copy behaviors


When we want to enable our `smart_pointer` to be pointed, things become a little bit complicated.

How do we define copy behavior? Basically, there are three reasonable behaviors of copy:

1. When we copy a pointer, copy the underlying object and return a new pointer points to it.
2. When we copy a pointer, transfer the ownership of the underlying objects, that is, the old pointer will not hold the object anymore.
3. Use a variable to record how many pointers point to the same object. The record increments each time we copy a pointer and decrements each time we deconstruct a pointer. The object will not be destroyed until the record is decreased to zero.

## First behavior - We don't copy the underlying object

Usually, we don't want to copy the underlying object when we copy a pointer for two reasons:
1. In this case, copy the object itself instead of the pointer is more suitable because one main reason for using pointers is to avoid copies of objects.
2. Moreover, if the pointer points to a derived class, we can't create a derived class with only base class. For example, suppose we use template to implement smart_ptr and

```cpp
template<class T>
class smart_ptr{...};
// suppose Circle is a derived class of Shape.
smart_ptr<Shape> circle = new Circle();
```

In the internal of smart_ptr, the template parameter T is `Shape`, smart_ptr<Shape> doesn't know the information of `Circle`, therefore we can't create an instance of Circle only with T.

Thus, we don't implement the first behavior in the code.

An pointer behaves as the reason2 indicates is a `unique_ptr` and behaves as the reason3 indicates is a `shared_ptr`, both are provided in C++11 standard.

## Second behavior - when to copy? 

### auto_ptr

In [1]:
%%file smart_ptr.h
#include <iostream>
template<class T>
class smart_pointer
{
    public:
        smart_pointer(T* ptr=NULL): ptr_(ptr) { }
        ~smart_pointer()
        {
            delete ptr_;
        }
        smart_pointer(smart_pointer& obj) // copy constructor
        {
                ptr_ =  obj.release();
        }
        smart_pointer& operator=(smart_pointer& obj) // copy assignment
        {
            smart_pointer copy = obj; // call copy constructor
            std::swap(copy, *this);
            return *this;
        }
        T* release()
        {
            T* ptr = ptr_;
            ptr_ = NULL;
            return ptr;
        }
    
        T* get() const { return ptr_; }
        // Operations to immitate pointers
        T& operator*() const { return *ptr_; }
        T* operator->() const { return ptr_;  }
        operator bool() const { return ptr_; }
    private:
        T* ptr_;
    
};

Overwriting smart_ptr.h


In [2]:
%%file test.cpp
#include "smart_ptr.h"
#include <string>
#include <iostream>

int main()
{
    smart_pointer<std::string> ptr(new std::string("hello world"));
    std::cout << ptr->c_str() << std::endl;
    std::cout << *ptr << std::endl;
    return 0;
}

Overwriting test.cpp


In [3]:
!g++ test.cpp -o test && ./test

hello world
hello world


Previous code seems to work well. However, such implementation encourages unexpected ownership transfers, leading to unexpected errors.

In [4]:
%%file test.cpp
#include "smart_ptr.h"
#include <string>
#include <iostream>

void print(smart_pointer<std::string> ptr)
{
    std::cout << ptr->c_str() << std::endl;
    std::cout << *ptr << std::endl;
}

int main()
{
    smart_pointer<std::string> ptr(new std::string("hello world"));
    print(ptr);
    print(ptr);
    return 0;
}

Overwriting test.cpp


In [5]:
!g++ test.cpp -o test && ./test

hello world
hello world
Segmentation fault (core dumped)


The main differences are that we move the print part into a `print` function and call the function twice. This code can be compiled and causes segmentation fault in the runtime.

The point is that the function `print` receives an `smart_pointer<std::string>` parameter, which invokes copy constructor and thus the ownership of the obj is transferred into the temporary obj created within the function. After the first `print` is called, the string obj on heap is already destructed as the temporary object is destructed. Thus, `ptr` now references a dead pointer and the second `print` fails without dubt.

The idea of this `smart_ptr` class is actually implemented by C++ as `auto_ptr`. 

### unique_ptr

The problem of `auto_ptr` exposed in the previous code can be addressed by introducing move semantic. C++ deprecates `auto_ptr` and provides `unique_ptr` to as a substitute.

The main problem of `auto_ptr` is to transfer ownership via copies. `unqiue_ptr` moves transferring ownership part into move constructor and move assignment operator. Here we do small changes on the basis of the previous code and implement a simple version of `unique_ptr`

In [6]:
%%file smart_ptr.h
#include <iostream>
template<class T>
class smart_pointer
{
    public:
        smart_pointer(T* ptr=NULL): ptr_(ptr) { }
        ~smart_pointer()
        {
            delete ptr_;
        }
        smart_pointer(smart_pointer&& obj) // copy constructor
        {
                ptr_ = obj.release();
        }
        smart_pointer& operator=(smart_pointer&& obj) // copy assignment
        {
            std::swap(obj, *this);
            return *this;
        }
        T* release()
        {
            T* ptr = ptr_;
            ptr_ = NULL;
            return ptr;
        }
    
        T* get() const { return ptr_; }
        // Operations to immitate pointers
        T& operator*() const { return *ptr_; }
        T* operator->() const { return ptr_;  }
        operator bool() const { return ptr_; }
    private:
        T* ptr_;
    
};

Overwriting smart_ptr.h


In [7]:
%%file test.cpp
#include "smart_ptr.h"
#include <string>
#include <iostream>

int main()
{
    smart_pointer<std::string> ptr(new std::string("hello world"));
    std::cout << ptr->c_str() << std::endl;
    std::cout << *ptr << std::endl;
    return 0;
}

Overwriting test.cpp


In [8]:
!g++ test.cpp -o test && ./test

hello world
hello world


Here, The only thing we did is to change the copy constructor and copy assignment operator into move constructor and move assignment operator.

## Third Behavior - Sharing ownership

### shared_ptr

This following code is still flawed.

In [44]:
%%file smart_ptr.h
#include <iostream>
class smart_count 
{
    public: 
        smart_count(int n_=0): n(n_) {}
        void decrement() { n--; }
        void increment() { n++; }
        int get() { return n; }
    private:
        int n;
    
};
template<class T>
class smart_pointer
{
    public:
        smart_pointer(T* ptr=NULL): ptr_(ptr) { 
            if (ptr_)
                count = new smart_count(1);
            else
                count = new smart_count(0);
        }
        ~smart_pointer()
        {
            count->decrement();
            if (count->get() == 0)
            {
                delete ptr_;
                delete count;
            }
        }
    
        smart_pointer(const smart_pointer& obj) // copy constructor
        {
                std::cout << "smart_pointer(const smart_pointer& obj)" << std::endl;
                ptr_ = obj.ptr_; // auto conversion
                count = obj.count;
                count->increment();
        }
        template<class U>
        smart_pointer(const smart_pointer<U>& obj) // conversion
        {
                std::cout << "smart_pointer(const smart_pointer<U>& obj)" << std::endl;
                ptr_ = obj.ptr_; // auto conversion
                count = obj.count;
                count->increment();
        }
    
        // here pass argument by value instead of by reference.
        smart_pointer& operator=(smart_pointer obj) // copy assignment
        {
            std::swap(obj, *this);
            count->increment();
            return *this;
        }
        T* get() const { return ptr_; }
        // Operations to immitate pointers
        T& operator*() const { return *ptr_; }
        T* operator->() const { return ptr_;  }
        operator bool() const { return ptr_; }
        int use_count() const { return count->get(); }
        template <class U>
        friend class smart_pointer;
    private:
        smart_count* count;
        T* ptr_;
};

Overwriting smart_ptr.h


In [40]:
%%file test.cpp
#include "smart_ptr.h"
#include <string>
#include <iostream>

class shape {
public:
    virtual ~shape() {};
};
class circle: public shape
{
    public:
        ~circle() { puts("~circle()");}
};

int main()
{
    smart_pointer<circle> ptr1(new circle());
    printf("#pointers = %d\n", ptr1.use_count());
    smart_pointer<shape> ptr2 = ptr1;
    printf("#pointers = %d\n", ptr2.use_count());
    smart_pointer<shape> ptr3;
    printf("#pointers = %d\n", ptr3.use_count());
    //     ptr3 = ptr2; 
    //     printf("#pointers = %d\n", ptr3.use_count());
    return 0;
}

Overwriting test.cpp


In [42]:
!g++ test.cpp -o test && ./test

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

